# Load data structures

In [1]:
load Par.mat
Par
Par.M
Par.P


Par = 

  struct with fields:

              P: [1x1 struct]
              M: [1x1 struct]
              B: [358x1 double]
            fas: [358x1 double]
         brange: [358x1 double]
          bminM: [358x1 double]
          bmaxM: [358x1 double]
          bminP: [358x1 double]
          bmaxP: [358x1 double]
         trange: [358x1 double]
          tminM: [358x1 double]
          tmaxM: [358x1 double]
          tmedM: [358x1 double]
          taveM: [358x1 double]
          tminP: [358x1 double]
          tmaxP: [358x1 double]
          tmedP: [358x1 double]
          taveP: [358x1 double]
         Phylum: {358x1 cell}
    Species_unq: {358x1 cell}
        RefOrig: {1x358 cell}
        RefComp: {1x358 cell}
      flag_LC50: [358x1 double]
       flag_del: [358x1 logical]
        flag_em: [358x1 logical]
       flag_eps: [358x1 logical]
        flag_eo: [358x1 logical]
            Iem: [358x1 logical]
            Ieo: [358x1 logical]
            In2: [41x1 double]
        dEdTbar

In [2]:
load Pobis.mat
Pobis


Pobis = 

  struct with fields:

        Species_unq: {358x1 cell}
       num_tot_obis: [1x358 double]
       num_woa_cell: [1x358 double]
        num_woa_unq: [1x358 double]
    Nobs_fracbelow1: [358x1 double]
        Phi_histWOA: [358x23 double]
           phi_wmax: [358x1 double]
          phi_wmaxM: [358x1 double]
           phi_wmin: [358x1 double]
          phi_wminM: [358x1 double]
              Zhist: [358x23 double]
              Ohist: [358x23 double]
              Thist: [358x23 double]
           T_atPatm: [358x23 double]
         FratioT_4d: [358x1 double]
         FratioO_4d: [358x1 double]
        FratioT_ssp: [358x3 double]
       FratioT_sspm: [358x3 double]
        FratioO_ssp: [358x3 double]
       FratioO_sspm: [358x3 double]
          phic_hist: [358x2 double]
           phic_f4d: [358x3 double]
          phic_fssp: [358x3 double]
         phic_fsspm: [358x3 double]
           phi_crit: [358x1 double]



Confirm correspondence in species list

In [3]:
N = length(Par.Species_unq);
for i = 1:N
    if Pobis.Species_unq{i} ~= Par.Species_unq{i}
      fprintf('mismatch row %d\n', i)
    end
end

## Extract parameters

Metabolic parameters

In [4]:
Ieo = isfinite(Par.P.est(:, 3)) & isfinite(Par.P.est(:, 1)) & isfinite(Pobis.phi_crit(:, 1));
fprintf('Number of taxa: %d', sum(Ieo))

Number of taxa: 61

In [5]:
del_norm = 0.25; % intra-specific B normalization (0 or 0.25)
alphaD = Par.M.est(Ieo, 1) .* Par.B(Ieo) .^ del_norm; 
alphaD_altB = Par.M.est(Ieo, 1) .* Par.B(Ieo) .^ 0.25; 
del = Par.M.est(Ieo, 2); 
Emet = Par.M.est(Ieo, 3); 
alphaD_err = Par.M.err(Ieo, 1); 
Emet_err = Par.M.err(Ieo, 3); 

Metabolic index parameters

In [9]:
Species_unq = Par.Species_unq(Ieo);
Phylum = Par.Phylum(Ieo);

Ao = Par.P.est(Ieo, 1); 
    
eps = Par.P.est(Ieo, 2); 
Eo = Par.P.est(Ieo, 3); 
Ao_err = Par.P.err(Ieo, 1); 
Eo_err = Par.P.err(Ieo, 3); 

Phi_crit = Pobis.phi_crit(Ieo, 1);

Vh = 1 ./ Ao;
Ac = Ao ./ Phi_crit; 

Esup = Emet - Eo;
alphaS = alphaD .* Ao;
alphaSbar = nanmean(alphaS);
tmp = ((alphaD_err ./ alphaD) .^2 + (Ao_err ./ Ao) .^2) .^ 0.5;
alphaS_err = tmp .* alphaS;

## Write JSON file

Write a json file that includes data and attributes for each field.

The format of this file looks like this:
```python
DataFrame = {
    field: {data: [...], 
            attrs: {"long_name": "Description...", 
                    "units": "Units string"
                   }
}
```

In [11]:
DataFrame = struct();

DataFrame.Species.data = Species_unq;
DataFrame.Species.data_type = 'string';
DataFrame.Species.attrs.long_name = 'Species';
DataFrame.Species.attrs.units = '';

DataFrame.Phylum.data = Phylum;
DataFrame.Phylum.data_type = 'string';
DataFrame.Phylum.attrs.long_name = 'Phylum';
DataFrame.Phylum.attrs.units = '';

DataFrame.alphaD_log10.data = log10(alphaD);
DataFrame.alphaD_log10.data_type = 'float';
DataFrame.alphaD_log10.attrs.long_name = 'log10(alpha_D)';

DataFrame.Emet.data = Emet;
DataFrame.Emet.data_type = 'float';
DataFrame.Emet.attrs.long_name = 'Temperature sensitivity of metabolism';
DataFrame.Emet.attrs.units = 'eV';

DataFrame.Eo.data = Eo;
DataFrame.Eo.data_type = 'float';
DataFrame.Eo.attrs.long_name = 'Temperature sensitivity';
DataFrame.Eo.attrs.units = 'eV';

DataFrame.Ao.data = Ao;
DataFrame.Ao.data_type = 'float';
DataFrame.Ao.attrs.long_name = 'Hypoxic tolerance';
DataFrame.Ao.attrs.units = '1/atm';

DataFrame.Ac.data = Ac;
DataFrame.Ac.data_type = 'float';
DataFrame.Ac.attrs.long_name = 'Hypoxic tolerance (normalized by $\\Phi_{crit}$)';
DataFrame.Ac.attrs.units = '1/atm';

DataFrame.Phi_crit.data = Phi_crit;
DataFrame.Phi_crit.data_type = 'float';
DataFrame.Phi_crit.attrs.long_name = '$\\Phi_{crit}$';
DataFrame.Phi_crit.attrs.units = '';


data_fields = fields(DataFrame);
sizes = [];
for i = 1:length(data_fields)
  sizes = [sizes; length(DataFrame.(data_fields{i}).data)];
end
if all(sizes ~= sizes(1))
  fprintf('DataFrameConstructor: size mismatch:\n')
  for i = 1:length(data_fields)
      fprintf('\t%s: %d\n', data_fields{i}, sizes(i))
  end
  error('')  
end


fid = fopen('MI-traits-Deutsch-etal-2020.json', 'w');
fprintf(fid, jsonencode(DataFrame, 'ConvertInfAndNaN', true));
fclose(fid);